In [1]:
!pip install keras==2.1.1

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
!pip install tensorflow==1.4.0

You are using pip version 18.1, however version 19.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


### Upcoming Modifications

In [4]:
# Training Change: Add Rests as notes
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
 
    elif isinstance(element,chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
        
    elif isinstance(element,note.Rest):
        notes.append(str('R'))
# # Training Change: Add Start/End Markers

# # Append the pitch of every note
# notes.append('start')
# for element in notes_to_parse:
#     ...
#     #stuff
# notes.append('end')

# Feature Add: Duration
note = notes_to_parse[7]
print('Note Length:', note.duration.type)
rest = notes_to_parse[6]
print('Rest Length:', rest.duration.type)

# Feature Add: Offset Change
# Is this a feature? Or is this inherent to duration (i.e I can just add this appropriately afterwards?)
# ANSWER: Doesn't look like it
last_offset = 0
offsets = []
for element in notes_to_parse:
    if isinstance(element, note.Note):
        notes.append(str(element.pitch))
 
    elif isinstance(element,chord.Chord):
        notes.append('.'.join(str(n) for n in element.normalOrder))
        
    elif isinstance(element,note.Rest):
        notes.append(str('R'))
    
    offset_delta = element.offset - last_offset
    if offset_delta < 0:
        print(offset_delta)
        break
    offsets.append(offset_delta)
    last_offset = element.offset

... 
x = note.Note('A3')
x.offset = 0.5

from collections import Counter
Counter(offsets)

# Model Change: Stateful
...

## Start

In [5]:
import pkg_resources
pkg_resources.require('tensorflow==1.4.0')
pkg_resources.require('keras==2.1.1')
import keras

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [6]:
import glob
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.utils import to_categorical
from keras import Sequential
from keras.layers import LSTM, Dropout, Dense, Activation
from keras.callbacks import ModelCheckpoint

In [8]:
notes = []
durations = []
pp = 0
for file in glob.glob("midi_songs/*.mid"):
    midi = converter.parse(file)
    notes_to_parse = None
    
    parts = instrument.partitionByInstrument(midi)
    
    if parts: # file has instrument parts
        notes_to_parse = parts.parts[0].recurse()
    else:
        notes_to_parse = midi.flat.notes
        
    # Append the pitch and duration of every note
    for element in notes_to_parse:
        if isinstance(element, note.Note) or isinstance(element,chord.Chord) or isinstance(element,note.Rest):
            
            # music21 can't handle notes beyond a certain speed. This skips them
            # In the original run, there was only 1 note in 2 files that broke it
            try:
                note_length = element.duration.type
            except: # This is naked because the except raised by music21 is custom 
                print(file)
                continue
            
            if isinstance(element, note.Note):
                notes.append(str(element.pitch))
            # If chord, encode ID of every note in cord separated by .
            elif isinstance(element,chord.Chord):
                chord_notes = [str(n) for n in element.pitches]
                notes.append(chord_notes)
            elif isinstance(element,note.Rest):
                # Sorry, I'm going to clear out 16th note rests because there are x8 more than
                # any other note
                if note_length != '16th':
                    notes.append('R-'+note_length)
                else:
                    continue
            else:
                raise TypeError('Invalid Note:{}'.format(element))
                break
            durations.append(note_length)    

midi_songs/Cids.mid
midi_songs/JENOVA.mid


In [175]:
# There are 88 keys, so 88 classes. There are also 12 types of rests, resulting in exactly
# 100 classes
NUM_KEYS = 88
NUM_RESTS = 12
NUM_CLASS = NUM_KEYS + NUM_RESTS

note_dict = {
        'A':0,
        'B':2,
        'C':3,
        'D':5,
        'E':7,
        'F':8,
        'G':10
}

reverse_note_dict = {v: k for k, v in note_dict.items()}

rest_dict = {
    'duplex-maxima':0,
    'maxima':1,
    'longa':2,
    'breve':3,
    'whole': 4,
    'half': 5,
    'quarter':6,
    'eighth':7,
    '16th':8,
    'zero':9,
    'complex':10,
    'inexpressible':11
}

reverse_rest_dict = {v: k for k, v in rest_dict.items()}

def note_encoder(note21, return_index=False):
    import re
    one_hot_matrix = np.zeros(NUM_CLASS)
    
    # If the note is a rest:
    if 'R' in note21:
        if ('32nd' in note21) or ('duplex' in note21):
            rest_length = 'complex'
        else:
            rest_length = re.findall(f'R-([a-z]*)',note21)[0]

        indx = NUM_KEYS + rest_dict[rest_length]
        
    # If a note:
    else:
        octave = int(re.findall(f'[0-9]',note21)[0])
        note = re.findall(f'[A-G]',note21)[0]

        if octave == 0:
            indx = note_dict[note]
        else: 
            indx = note_dict[note]+12*(octave-1)

        if len(note21) != 2: #There's a flat or sharp
            if '#' in note21:
                indx+=1
            elif '-' in note21:
                indx-=1
            else:
                raise TypeError('Unkown string input')
    one_hot_matrix[indx] = 1
    
    if return_index:
        return(indx)
    else:
        return(one_hot_matrix)

def one_hot_note_encoder(note21):
    if isinstance(note21,str):
        one_hot_matrix = note_encoder(note21)
    elif isinstance(note21,list):
        one_hot_matrix = np.zeros(NUM_CLASS)
        for note in note21:
            one_hot_matrix+=note_encoder(note)
            
    return(one_hot_matrix)

# Function added for 6 Feature; return_index flag also added to be lazy
def note_to_int(note21):
    note_int = []
    if isinstance(note21,str):
        note_int.append(note_encoder(note21,return_index=True))
    elif isinstance(note21,list):
        note_int = []
        for chord_note in note21[:6]:
            note_int.append(note_encoder(chord_note,return_index=True))
       
    while len(note_int) < 6:
        note_int.append(0)
        
    return(note_int)

def note_decoder(indx):    
    
    if indx > NUM_KEYS:
        decoded = reverse_rest_dict[indx-NUM_KEYS]
    else:
        decoded = []
        notes_played = np.argwhere(encoded_notes[i] == np.amax(encoded_notes[i]))
        for note in notes_played:
            t = note[0]
            # print(t)
            octave = 1
            accent = ''
            while t > 11:
                t -= 12
                octave += 1

            if (t == 4) or (t == 11) or (t == 9):
                accent = '#'
                t-= 1

            elif (t == 1) or (t == 6):
                accent = '-'
                t += 1

            guess = reverse_note_dict[t] + accent + str(octave)
            decoded.append(guess)
        if len(decoded) == 1:
            decoded = decoded[0]
            
    return(decoded)
    

In [50]:
ma_list = []
for i in notes:
    if isinstance(i,list):
        ma_list.append(len(i))
Counter(ma_list)

Counter({2: 18614, 4: 732, 3: 4369, 5: 47, 6: 18, 7: 8, 8: 2})

## TODO

- Encoder/Decoder for multiclass keyboarding **[DONE]**
- 100 feature input **[DONE/FAILED]**
- 6 Note capped feature input (for guitar strings) ** IN PROGRESS **
- Starts/Ends || End predictor machine (logress, with boost to precision)
- Masking layer to get rid of the nans

## 100 features Test

In [35]:
# create input/output sequences -- output is next note 

sequence_length = 100

network_input = []
network_output = []
network_output_note = []
network_output_duration = []

encoded_notes = [one_hot_note_encoder(note) for note in notes]

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length,1):
    sequence_in = encoded_notes[i:i + sequence_length]
#     duration_in = durations[i:i+sequence_length]
    
    sequence_out = encoded_notes[i + sequence_length]
#     duration_out = durations[i + sequence_length]

    network_input.append(sequence_in)
    network_output.append(sequence_out)
    
network_output_np = np.array(network_output)
network_input_np = np.array(network_input)

n_patterns = len(network_input)
print(n_patterns, 'training examples')

# reshape the input into a format compatible with LSTM layers

## TODO: I now have 100 features (?) for every note + rest style. Let's see if this works
network_input = np.reshape(network_input_np, (n_patterns, sequence_length,NUM_CLASS))
network_output = network_output_np

85765 training examples


In [36]:
network_input.shape, network_output.shape

((85765, 100, 100), (85765, 100))

In [44]:
from keras.models import Model
from keras.models import Input

# Rewritten with Keras' functional API
model_input = Input(shape=(network_input.shape[1], network_input.shape[2]))

# layers = LSTM(NUM_CLASS,return_sequences=True)(model_input)
# layers = Dropout(0.3)(layers)
# layers = LSTM(20)(layers)

layers = LSTM(NUM_CLASS)(model_input)

output_note = Dense(NUM_CLASS)(layers)
model = Model(inputs=model_input, outputs=output_note)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [45]:
filepath = "weights/test_weights_v2_0_100f.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=10, batch_size=256, callbacks=callbacks_list)

Epoch 1/10
85765/85765 [==============================] - 88s 1ms/step - loss: 11.6020
Epoch 2/10
85765/85765 [==============================] - 83s 969us/step - loss: nan
Epoch 3/10
85765/85765 [==============================] - 9142s 107ms/step - loss: nan
Epoch 4/10
81920/85765 [===========================>..] - ETA: 3s - loss: nan

KeyboardInterrupt: 

## 6 Features

In [162]:
# create input/output sequences -- output is next note 

sequence_length = 100

network_input = []
network_output = []
network_output_note = []
network_output_duration = []

encoded_notes = [one_hot_note_encoder(note) for note in notes]
int_notes = [note_to_int(note) for note in notes]

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length,1):
    
    sequence_in = int_notes[i:i + sequence_length]
    sequence_out = encoded_notes[i + sequence_length]

    network_input.append(sequence_in)
    network_output.append(sequence_out)
    
network_output_np = np.array(network_output)
network_input_np = np.array(network_input)

n_patterns = len(network_input)
print(n_patterns, 'training examples')

# reshape the input into a format compatible with LSTM layers

## I now have 6 features for max 6 note cords
network_input = np.reshape(network_input_np, (n_patterns, sequence_length,6))
network_output = network_output_np

85765 training examples


In [168]:
from keras.models import Model
from keras.models import Input

# Rewritten with Keras' functional API
model_input = Input(shape=(network_input.shape[1], network_input.shape[2]))

# layers = LSTM(NUM_CLASS,return_sequences=True)(model_input)
# layers = Dropout(0.3)(layers)
# layers = LSTM(20)(layers)

layers = LSTM(sequence_length)(model_input)

output_note = Dense(NUM_CLASS)(layers)
model = Model(inputs=model_input, outputs=output_note)
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [170]:
filepath = "weights/test_weights_v2_0_6f.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, network_output, epochs=5, batch_size=256, callbacks=callbacks_list)

Epoch 1/5
85765/85765 [==============================] - 57s 667us/step - loss: 11.5974
Epoch 2/5
85765/85765 [==============================] - 58s 678us/step - loss: 11.3151
Epoch 3/5
85765/85765 [==============================] - 57s 665us/step - loss: 11.2911
Epoch 4/5
85765/85765 [==============================] - 57s 668us/step - loss: 11.3698
Epoch 5/5
85765/85765 [==============================] - 57s 667us/step - loss: 11.3307


In [205]:
# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

In [209]:
prediction_output = []

# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

# generate 500 notes
for note_index in range(100):
    
    # Reshape and scale sequence
#     prediction_input = np.reshape(pattern, (1,pattern.shape[0],pattern.shape[1]))
    prediction_input = np.reshape(pattern, (1, sequence_length,6))
    
    # Predict next note
    note_prediction = model.predict(prediction_input, verbose=0)
    
    # Best Note:
    note_draw = np.argmax(note_prediction)
    note_str = note_decoder(note_draw)
    
    if isinstance(note_draw,list):
        note_list = note_draw
    else:
        note_list = []
        note_list.append(note_draw)
    while len(note_list) < 6:
        note_list.append(0)
    
    # Add note to our new sequence 
    
    new_pattern = prediction_input
    new_pattern = np.append(new_pattern,note_list)
    new_pattern = np.delete(new_pattern,[i for i in range(6)]).reshape(100,6)
    
    
    pattern = new_pattern
    
    prediction_output.append(note_str)
prediction_output

['C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4',
 'C4']

# Old Code

In [11]:
from collections import Counter
print(Counter(durations))
duration_dict = {
    'duplex-maxima':16.0,
    'maxima':8.0,
    'longa':4.0,
    'breve':2.0,
    'whole': 1.0,
    'half': 0.5,
    'quarter':0.25,
    'eighth':0.125,
    '16th':0.0625,
    'zero':0.0,
    'complex':0.03125,
    'inexpressible':0.0155
    
    
}

Counter({'eighth': 36476, '16th': 32447, 'quarter': 7575, 'zero': 3777, 'half': 2871, 'complex': 1845, 'whole': 532, 'inexpressible': 119, '32nd': 116, 'breve': 59, 'longa': 31, 'maxima': 9, 'duplex-maxima': 8})


In [13]:
sorted_by_value = sorted(Counter(notes).items(), key=lambda kv: kv[1])
sorted_by_value[-10:]

[('E-5', 1675),
 ('3.7', 1795),
 ('E-3', 1855),
 ('C3', 1877),
 ('F5', 1972),
 ('D3', 2038),
 ('D4', 2189),
 ('B-2', 2338),
 ('C4', 2345),
 ('B-3', 2776)]

In [14]:
# Convert string notes into integers for stronger LSTM performance

# get all pitch names
pitchnames = sorted(set(item for item in notes))
durationnames = sorted(set(item for item in durations))

# create a dictionary to map pitches to integers
note_to_int = dict((note,number) for number, note in enumerate(pitchnames))
duration_to_int = dict((duration,number) for number, duration in enumerate(durationnames))

In [16]:
note_to_int['R-quarter']

367

In [17]:
n_vocab = len(note_to_int)
n_duration = len(duration_dict)
from sklearn.preprocessing import OneHotEncoder

# create input/output sequences -- output is next note 

sequence_length = 100

network_input = []
network_output = []
network_output_note = []
network_output_duration = []

# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length,1):
    sequence_in = notes[i:i + sequence_length]
    duration_in = durations[i:i+sequence_length]
    
    sequence_out = notes[i + sequence_length]
    duration_out = durations[i + sequence_length]
    
    sequence_in = [note_to_int[char] for char in sequence_in]
    duration_in = [duration_to_int[char] for char in duration_in]
    
    zipped_in = [[note, duration] for note,duration in zip(sequence_in,duration_in)]
#     zipped_out = [note_to_int[sequence_out],duration_out]
    
    network_input.append(zipped_in)
#     network_output.append(note_to_int[sequence_out])
#     network_output_note.append(zipped_out)
    network_output_note.append(note_to_int[sequence_out])
    network_output_duration.append(duration_to_int[duration_out])
    
network_output_note = np.array(network_output_note)
network_output_note = to_categorical(network_output_note)

network_output_duration = np.array(network_output_duration)
network_output_duration = to_categorical(network_output_duration)

# network_output = np.array([[i,j] for i,j in zip(network_output_note,network_output_duration)])

n_patterns = len(network_input)
print(n_patterns, 'training examples')

# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length,2))

print('Input Shape:', network_input.shape)
print('Output #1 Shape:', network_output_note.shape)
print('Output #1 Shape:', network_output_duration.shape)

# normalize input
n_vocab = len(note_to_int)
n_duration = len(durationnames)
# network_input = network_input / float(n_vocab)

# network_output = to_categorical(network_output)
# network_output = np.reshape(network_output, (network_output.shape[0], 1, network_output.shape[1]))

85765 training examples
Input Shape: (85765, 100, 2)
Output #1 Shape: (85765, 369)
Output #1 Shape: (85765, 13)


In [ ]:
# network_input[2]

In [55]:
from keras.models import Model
from keras.models import Input

# Rewritten with Keras' functional API
model_input = Input(shape=(network_input.shape[1], network_input.shape[2]))

layers = LSTM(88,return_sequences=True)(model_input)
layers = Dropout(0.3)(layers)
layers = LSTM(20)(layers)


output_note = Dense(n_vocab)(layers)
output_duration = Dense(n_duration)(layers)

model = Model(inputs=model_input, outputs=[output_note,output_duration])
model.compile(loss=['categorical_crossentropy','categorical_crossentropy'], 
              optimizer='rmsprop',loss_weights = [1.0,0.5])

In [61]:
output_duration

<tf.Tensor 'dense_24/BiasAdd:0' shape=(?, 13) dtype=float32>

In [66]:
note_model = Sequential()
note_model.add(LSTM(20,  
               input_shape=(network_input.shape[1], network_input.shape[2]),
               return_sequences=True))
note_model.add(LSTM(20))
note_model.add(Dense(n_vocab))
note_model.add(Activation('softmax'))
note_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [138]:
filepath = "weights/test_weights_v2_0_2_notes.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
note_model.fit(network_input, network_output_note, \
          epochs=10, batch_size=256, callbacks=callbacks_list)

Epoch 1/10
85765/85765 [==============================] - 52s 609us/step - loss: 3.3252
Epoch 2/10
85765/85765 [==============================] - 52s 610us/step - loss: 3.2758
Epoch 3/10
85765/85765 [==============================] - 52s 601us/step - loss: 3.2427
Epoch 4/10
85765/85765 [==============================] - 52s 610us/step - loss: 3.2962
Epoch 5/10
85765/85765 [==============================] - 52s 610us/step - loss: 3.2406
Epoch 6/10
85765/85765 [==============================] - 50s 588us/step - loss: 3.3594
Epoch 7/10
85765/85765 [==============================] - 52s 604us/step - loss: 3.2348
Epoch 8/10
85765/85765 [==============================] - 51s 591us/step - loss: 3.2166
Epoch 9/10
85765/85765 [==============================] - 50s 583us/step - loss: 3.2358
Epoch 10/10
85765/85765 [==============================] - 50s 586us/step - loss: 3.2245


In [69]:
duration_model = Sequential()
duration_model.add(LSTM(20,  
               input_shape=(network_input.shape[1], network_input.shape[2]),
               return_sequences=True))
duration_model.add(LSTM(20))
duration_model.add(Dense(n_duration))
duration_model.add(Activation('softmax'))
duration_model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [139]:
filepath = "weights/test_weights_v2_0_2_durations.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
duration_model.fit(network_input, network_output_duration, \
          epochs=5, batch_size=256, callbacks=callbacks_list)

Epoch 1/5
85765/85765 [==============================] - 50s 579us/step - loss: 0.6585
Epoch 2/5
85765/85765 [==============================] - 51s 600us/step - loss: 0.6552
Epoch 3/5
85765/85765 [==============================] - 49s 570us/step - loss: 0.6547
Epoch 4/5
85765/85765 [==============================] - 49s 572us/step - loss: 0.6485
Epoch 5/5
85765/85765 [==============================] - 49s 570us/step - loss: 0.6459


In [58]:
filepath = "weights/test_weights_v2_0_2.hdf5"    
checkpoint = ModelCheckpoint(
    filepath, monitor='loss', 
    verbose=0,        
    save_best_only=True,        
    mode='min'
)    
callbacks_list = [checkpoint]     
model.fit(network_input, [network_output_note,network_output_duration], \
          epochs=100, batch_size=256, callbacks=callbacks_list)

In [142]:
# Pick a random start sequence
start = np.random.randint(0, len(network_input)-1)
pattern = network_input[start]

# The reverse of note_to_int
int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
int_to_duration = dict((number, duration) for number, duration in enumerate(durationnames))

In [143]:
prediction_output = []

# generate 500 notes
for note_index in range(100):
    
    # Reshape and scale sequence
    prediction_input = np.reshape(pattern, (1,pattern.shape[0],pattern.shape[1]))
    
    # Predict next note
    note_prediction = note_model.predict(prediction_input, verbose=0)
    duration_prediction = duration_model.predict(prediction_input, verbose=0)
    
    # Best Note:
    note_draw = np.argmax(note_prediction)
    duration_draw = np.argmax(duration_prediction)
    
    result_draw = [note_draw,duration_draw]
    
    # Add note to our new sequence 
    note_result = int_to_note[note_draw]
    duration_result = int_to_duration[duration_draw]
    
    result = [note_result,duration_result]
    
    new_pattern = prediction_input
    new_pattern = np.append(new_pattern,result_draw)
    new_pattern = np.delete(new_pattern,[0,1]).reshape(100,2)
    
    pattern = new_pattern
    
    
    prediction_output.append(result)
prediction_output

[['2.5', '16th'],
 ['B-3', '16th'],
 ['D5', '16th'],
 ['E-5', '16th'],
 ['E-5', '16th'],
 ['F5', '16th'],
 ['2.5', '16th'],
 ['C#3', '16th'],
 ['2.5', '16th'],
 ['D3', 'zero'],
 ['2.7', 'zero'],
 ['D5', '16th'],
 ['5.10', 'zero'],
 ['7.8', '16th'],
 ['E-3', '16th'],
 ['3.7', '16th'],
 ['B-3', '16th'],
 ['G6', '16th'],
 ['G6', '16th'],
 ['7.10', '16th'],
 ['B-3', '16th'],
 ['R-32nd', '16th'],
 ['E-5', 'zero'],
 ['B-3', '16th'],
 ['G#6', '16th'],
 ['G#6', '16th'],
 ['G#6', '16th'],
 ['R-32nd', '16th'],
 ['C3', '16th'],
 ['G#6', '16th'],
 ['B-1', '16th'],
 ['B-1', '16th'],
 ['R-32nd', '16th'],
 ['F4', '16th'],
 ['F4', '16th'],
 ['F4', '16th'],
 ['C#4', '16th'],
 ['C5', '16th'],
 ['C5', '16th'],
 ['C5', '16th'],
 ['R-eighth', '16th'],
 ['D5', '16th'],
 ['F5', '16th'],
 ['F5', '16th'],
 ['F5', '16th'],
 ['F5', '16th'],
 ['G5', '16th'],
 ['B-4', '16th'],
 ['C5', '16th'],
 ['C5', '16th'],
 ['G5', '16th'],
 ['D5', '16th'],
 ['G#4', '16th'],
 ['G#4', '16th'],
 ['F5', '16th'],
 ['F5', '16th'],
 

In [145]:
offset = 0
output_notes = []

for pattern in prediction_output:
    next_note = pattern[0]
    next_duration = pattern[1]
    
    if ('.' in next_note) or next_note.isdigit():
        notes_in_chord = next_note.split('.')
        notes = []
        for current_note in notes_in_chord:
            single_note = note.Note(int(current_note))
            single_note.storedInstrument = instrument.Piano()
            notes.append(single_note)
        new_note = chord.Chord(notes)
    elif 'R' in next_note:
        new_note = note.Rest()
    # pattern is a note
    else:
        new_note = note.Note(next_note)
        new_note.storedInstrument = instrument.Piano()
    
    if next_duration != 'zero':
        new_note.duration.type = next_duration
 
        if next_duration == '16th':
            offset += 0.125*3
        elif next_duration == 'eighth':
            offset += 0.25*3
        elif next_duration == 'quarter':
            offset += 0.5*3
        elif next_duration == 'half':
            offset += 1.0*3
        elif next_duration == 'whole':
            offset += 2.0*3
    
    new_note.offset = offset
    output_notes.append(new_note)

In [146]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='v2_0_2.mid')

'v2_0_2.mid'

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    note = patt
    
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    elif pattern == 'r':
        new_note = note.Rest()
        
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [44]:
prediction_output = []

# generate 500 notes
for note_index in range(100):
    
    # Reshape and scale sequence
    prediction_input = np.reshape(pattern, (1,pattern.shape[0],pattern.shape[1]))
    
    # Predict next note
    prediction = model.predict(prediction_input, verbose=0)
    
    
    # Best Note:
    note_draw = np.argmax(prediction[0])
    duration_draw = np.argmax(prediction[1])
    
    result_draw = [note_draw,duration_draw]
    
    # Add note to our new sequence 
    note_result = int_to_note[note_draw]
    duration_result = int_to_duration[duration_draw]
    
    result = [note_result,duration_result]
    
    new_pattern = prediction_input
    new_pattern = np.append(new_pattern,result_draw)
    new_pattern = np.delete(new_pattern,[0,1]).reshape(100,2)
    
    pattern = new_pattern
    
    
    prediction_output.append(result)
    

In [45]:
prediction_output

[['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '16th'],
 ['0', '

In [ ]:
import matplotlib.pyplot as plt
plt.hist(prediction.flatten())
plt.show()

In [ ]:
offset = 0
output_notes = []

# create note and chord objects based on the values generated by the model
for pattern in prediction_output:
    
    # pattern is a chord
    if ('.' in pattern) or pattern.isdigit():
        notes_in_chord = pattern.split('.')
        notes = []
        for current_note in notes_in_chord:
            new_note = note.Note(int(current_note))
            new_note.storedInstrument = instrument.Piano()
            notes.append(new_note)
        new_chord = chord.Chord(notes)
        new_chord.offset = offset
        output_notes.append(new_chord)
        
    elif pattern == 'r':
        new_note = note.Rest()
        
    # pattern is a note
    else:
        new_note = note.Note(pattern)
        new_note.offset = offset
        new_note.storedInstrument = instrument.Piano()
        output_notes.append(new_note)
        
    # increase offset each iteration so that notes do not stack
    offset += 0.5

In [ ]:
midi_stream = stream.Stream(output_notes)
midi_stream.write('midi', fp='new_test_garbage_100.mid')

In [ ]:
# prediction_output

In [ ]:
pattern

In [ ]:
# pattern

In [ ]:
# network_input[2]

In [ ]:
# sequence_in

In [ ]:
model.summary()